<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d438acbe38eb2b53bdc44dc1064ff0aa5712494342cf303d871a8795d2817927
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-24 10:23:36
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: 10.94 K (0.08%)
Current PnL: -25.15 L (-16.24%)
CY Booked + Current PnL: -10.86 L (-7.01%)
-------------------
Total profit:  1.65 L
Total loss:  -26.80 L
-------------------
Total Booked + Current PnL: 16.12 L (12.61%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.07%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 89.78 L (63.27%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.05,-19.09,23.70,0.09,19325.0,-19236.0,81541.0,89.33,39.0,M-SC,2.62,253.0,-1.00,0.58,7.13,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.45,8.88,9.93,19.69,20731.0,17028.0,208770.0,-1.77,55.0,X-MC,2.48,78.0,0.82,1.48,16.28,XY25,NTT,AC
50,MASFIN,398.61,0.06,-1.04,23.33,22.05,22621.0,-1020.0,96960.0,-14.63,57.0,H-SC,6.66,164.0,-0.05,0.69,40.94,XR,ATH,FINANCE
43,ITC,452.00,0.18,0.02,10.76,10.78,25864.0,53.0,240371.0,-40.34,59.0,X-LC,2.01,5.0,0.00,1.70,5.26,X40,NTT,FMCG
52,NESTLEIND,1377.00,0.15,10.94,9.42,21.40,27739.0,29040.0,294466.0,2.90,56.0,X-LC,5.53,12.0,1.05,2.08,18.31,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,3.53,-37.75,81.37,12.90,110672.0,-82485.0,136011.0,100.72,56.0,H-SC,12.64,161.0,-0.75,0.96,13.09,OX40N,NTT,FINANCE
26,GILLETTE,11206.78,2.69,-2.70,36.09,32.42,89160.0,-6846.0,247050.0,-20.08,47.0,X-SC,10.71,85.0,-0.08,1.75,12.00,X40,ATH,FMCG
63,SFL,1287.00,2.65,-41.03,114.73,26.63,177409.0,-107586.0,154632.0,17.12,49.0,M-SC,12.31,239.0,-0.61,1.09,5.86,XY24,NTT,MISC
22,DIXON,18931.72,2.06,-12.79,44.56,26.07,75863.0,-24973.0,170248.0,-57.60,38.0,X-MC,12.37,56.0,-0.33,1.21,6.03,X40N,ATH,IT
15,CAMS,950.00,1.58,1.18,23.03,24.48,58686.0,2970.0,254826.0,-80.65,53.0,X-SC,1.27,86.0,0.05,1.80,24.59,X40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.10,-1.27,-38.67,127.65,39.61,80241.0,-39640.0,62860.0,-58.53,31.0,H-SC,31.09,135.0,-0.49,0.45,0.00,AR,ATH,MISC
75,TMPV,600.00,-1.07,-26.35,66.97,22.98,111424.0,-59519.0,166379.0,-23.08,50.0,X-LC,6.41,3.0,-0.53,1.18,4.64,XY24,NTT,AUTO
67,SONACOMS,806.63,-1.04,-15.53,65.19,39.54,55706.0,-15707.0,85452.0,-33.96,49.0,M-SC,3.87,220.0,-0.28,0.61,20.50,AR,ATH,AUTO
69,SURYODAY,216.00,-0.91,-18.55,51.05,23.03,74462.0,-33211.0,145860.0,57.89,58.0,H-SC,7.71,167.0,-0.45,1.03,44.72,XR,NTT,BANKS
28,GREENPANEL,537.00,-0.90,-36.65,125.95,43.14,146074.0,-67100.0,115978.0,159.69,43.0,M-SC,10.64,240.0,-0.46,0.82,9.09,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,1.4,-0.16,19.42,19.23,29173.0,-240.0,150220.0,16.96,76.0,L-LC,2.51,179.0,-0.01,1.06,21.08,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.33,-2.49,113.23,107.91,163008.0,-3682.0,143962.0,-20.55,53.0,M-SC,11.85,216.0,-0.02,1.02,7.34,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.31,-17.20,20.80,0.02,30063.0,-30025.0,144534.0,120.42,40.0,M-SC,11.37,234.0,-1.00,1.02,25.72,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.05,-19.09,23.70,0.09,19325.0,-19236.0,81541.0,89.33,39.0,M-SC,2.62,253.0,-1.00,0.58,7.13,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.52,-22.16,69.62,32.03,71443.0,-29216.0,102619.0,-15.70,60.0,H-SC,8.95,148.0,-0.41,0.73,14.94,OX40N,NTT,IT
66,SIS,528.00,0.56,-23.69,58.92,21.26,49919.0,-26308.0,84724.0,1999.48,52.0,H-SC,4.79,166.0,-0.53,0.60,14.47,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.74,10.49,55.91,72.27,104678.0,17778.0,187226.0,-5.17,69.0,M-LC,3.9,99.0,0.17,1.33,17.31,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.74,10.49,55.91,72.27,104678.0,17778.0,187226.0,-5.17,69.0,M-LC,3.90,99.0,0.17,1.33,17.31,XR,NTT,IT
25,FINCABLES,1641.55,-0.33,-2.49,113.23,107.91,163008.0,-3682.0,143962.0,-20.55,53.0,M-SC,11.85,216.0,-0.02,1.02,7.34,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.57,-3.44,114.09,106.72,135871.0,-4245.0,119091.0,-52.15,43.0,H-SC,4.48,139.0,-0.03,0.84,19.13,AR,ATH,MISC
6,ASIANTILES,137.00,-0.56,-3.28,87.83,81.67,79694.0,-3073.0,90737.0,8172.22,70.0,L-SC,12.86,271.0,-0.04,0.64,75.72,XR,NTT,CERAMICS
50,MASFIN,398.61,0.06,-1.04,23.33,22.05,22621.0,-1020.0,96960.0,-14.63,57.0,H-SC,6.66,164.0,-0.05,0.69,40.94,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.43,-18.25,90.17,55.47,179345.0,-44389.0,198897.0,-27.60,30.0,X-MC,15.61,64.0,-0.25,1.41,1.91,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.31,-18.34,75.51,43.32,100428.0,-29872.0,133000.0,-29.79,31.0,X-SC,5.98,90.0,-0.30,0.94,2.12,X40N,ATH,ELECTRICAL
8,AWL,485.00,-0.02,-24.26,102.67,53.50,234634.0,-73219.0,228532.0,-63.37,33.0,X-MC,5.89,58.0,-0.31,1.62,1.51,XY24,NTT,FMCG
3,ACC,3906.00,-0.64,-26.75,124.10,64.15,216306.0,-63651.0,174300.0,-57.19,34.0,X-SC,4.21,82.0,-0.29,1.23,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.09,-38.75,120.74,35.20,95159.0,-49857.0,78813.0,3.39,35.0,X-SC,17.01,92.0,-0.52,0.56,1.09,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.22,-2.86,24.71,21.14,54566.0,-6513.0,220826.0,-9.80,47.0,X-MC,1.03,74.0,-0.12,1.56,24.47,XY24,NTT,PAINTS
15,CAMS,950.0,1.58,1.18,23.03,24.48,58686.0,2970.0,254826.0,-80.65,53.0,X-SC,1.27,86.0,0.05,1.80,24.59,X40N,NTT,MISC
78,UNITDSPR,1644.0,-0.85,5.01,15.01,20.77,36906.0,11730.0,245874.0,-4.14,51.0,X-MC,1.97,62.0,0.32,1.74,12.27,X40N,NTT,BREWERIES
43,ITC,452.0,0.18,0.02,10.76,10.78,25864.0,53.0,240371.0,-40.34,59.0,X-LC,2.01,5.0,0.00,1.70,5.26,X40,NTT,FMCG
20,DABUR,735.0,0.19,-3.40,48.76,43.70,116847.0,-8430.0,239638.0,-11.44,37.0,X-MC,2.12,72.0,-0.07,1.70,11.14,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.64,-26.75,124.10,64.15,216306.0,-63651.0,174300.0,-57.19,34.0,X-SC,4.21,82.0,-0.29,1.23,0.00,XY24,BTT,CEMENT
19,COLPAL,3726.84,-0.21,-20.19,77.32,41.51,162511.0,-53185.0,210180.0,-4.98,39.0,X-MC,7.87,61.0,-0.33,1.49,0.69,XY25,ATH,FMCG
11,BATAINDIA,2096.00,-0.09,-38.75,120.74,35.20,95159.0,-49857.0,78813.0,3.39,35.0,X-SC,17.01,92.0,-0.52,0.56,1.09,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.02,-24.26,102.67,53.50,234634.0,-73219.0,228532.0,-63.37,33.0,X-MC,5.89,58.0,-0.31,1.62,1.51,XY24,NTT,FMCG
9,BAJAJHFL,181.50,-0.43,-18.25,90.17,55.47,179345.0,-44389.0,198897.0,-27.60,30.0,X-MC,15.61,64.0,-0.25,1.41,1.91,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.75,-27.71,97.60,42.84,45863.0,-18015.0,46991.0,-52.62,52.0,X-SC,33.80,83.0,-0.39,0.33,6.31,XY24,NTT,MISC
59,RELAXO,1176.00,0.95,-47.25,188.73,52.31,144514.0,-68588.0,76572.0,-43.86,47.0,X-SC,7.32,91.0,-0.47,0.54,2.50,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.09,-38.75,120.74,35.20,95159.0,-49857.0,78813.0,3.39,35.0,X-SC,17.01,92.0,-0.52,0.56,1.09,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.71,-1.51,25.97,24.08,32472.0,-1912.0,125038.0,-9.50,46.0,X-SC,6.44,89.0,-0.06,0.89,17.81,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.31,-18.34,75.51,43.32,100428.0,-29872.0,133000.0,-29.79,31.0,X-SC,5.98,90.0,-0.30,0.94,2.12,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.15,-8.85,32.43,20.71,111806.0,-33477.0,344760.0,-21.36,70.0,X-LC,3.20,1.0,-0.30,2.44,15.20,X40,ATH,IT
41,INFY,1972.00,-0.16,13.39,18.39,34.24,65247.0,41906.0,354794.0,-11.70,68.0,X-LC,6.17,2.0,0.64,2.51,22.78,X40,NTT,IT
75,TMPV,600.00,-1.07,-26.35,66.97,22.98,111424.0,-59519.0,166379.0,-23.08,50.0,X-LC,6.41,3.0,-0.53,1.18,4.64,XY24,NTT,AUTO
81,VBL,671.64,-0.74,-3.40,40.45,35.67,123410.0,-10750.0,305092.0,-14.83,54.0,X-LC,3.46,4.0,-0.09,2.16,9.98,X40N,ATH,FMCG
43,ITC,452.00,0.18,0.02,10.76,10.78,25864.0,53.0,240371.0,-40.34,59.0,X-LC,2.01,5.0,0.00,1.70,5.26,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.30,-37.06,111.14,32.89,54005.0,-28614.0,48592.0,-699.86,56.0,L-MC,6.82,259.0,-0.53,0.34,33.82,XR,NTT,BANKS
56,RAJESHEXPO,518.00,0.13,-56.86,131.96,0.07,78090.0,-78000.0,59177.0,2069.19,70.0,L-SC,4.06,268.0,-1.00,0.42,47.18,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-0.56,-3.28,87.83,81.67,79694.0,-3073.0,90737.0,8172.22,70.0,L-SC,12.86,271.0,-0.04,0.64,75.72,XR,NTT,CERAMICS
50,MASFIN,398.61,0.06,-1.04,23.33,22.05,22621.0,-1020.0,96960.0,-14.63,57.0,H-SC,6.66,164.0,-0.05,0.69,40.94,XR,ATH,FINANCE
13,BSOFT,831.70,-0.32,-17.62,83.03,50.79,96209.0,-24781.0,115872.0,5.53,73.0,H-SC,8.47,151.0,-0.26,0.82,34.91,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.32,-17.62,83.03,50.79,96209.0,-24781.0,115872.0,5.53,73.0,H-SC,8.47,151.0,-0.26,0.82,34.91,XR,ATH,IT
56,RAJESHEXPO,518.00,0.13,-56.86,131.96,0.07,78090.0,-78000.0,59177.0,2069.19,70.0,L-SC,4.06,268.0,-1.00,0.42,47.18,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-0.56,-3.28,87.83,81.67,79694.0,-3073.0,90737.0,8172.22,70.0,L-SC,12.86,271.0,-0.04,0.64,75.72,XR,NTT,CERAMICS
61,ROUTE,2227.21,-0.59,-46.06,213.38,69.05,153166.0,-61285.0,71781.0,-56.49,54.0,H-SC,20.62,140.0,-0.40,0.51,10.70,SR,ATH,IT
17,COALINDIA,484.83,1.40,-0.16,19.42,19.23,29173.0,-240.0,150220.0,16.96,76.0,L-LC,2.51,179.0,-0.01,1.06,21.08,XY25,ATH,MINING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.37
1,25,44.81
2,50,76.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.99
MC    30.69
LC    24.28
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.74
X40      23.04
X40N     12.94
XR        9.93
XY25      9.11
AR        8.91
OX40N     7.54
SR        1.00
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.18
H-SC    23.85
X-LC    20.78
M-SC    11.55
X-SC     8.04
H-MC     4.79
L-SC     1.55
M-MC     1.38
M-LC     1.33
H-LC     1.11
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.21,-7.30,41.80
IT,13.17,-13.15,70.05
FINANCE,9.57,-17.84,69.10
MISC,7.29,-25.93,77.45
ELECTRICAL,5.95,-11.62,52.50
PAINTS,5.79,-12.85,29.61
INSURANCE,4.74,-1.21,35.85
PHARMA,4.04,-2.83,35.20
AUTO,2.85,-30.38,75.58


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3142193.0,21
AR,1315019.0,10
XR,1259472.0,13
X40,967428.0,14
X40N,856553.0,9
OX40N,727455.0,10
XY25,353125.0,6
SR,279324.0,2
MH,77564.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3609176.0,25
M-SC,1425853.0,15
X-MC,1409975.0,16
X-LC,790254.0,11
X-SC,782588.0,8
H-MC,402506.0,3
L-SC,242764.0,3
M-LC,104678.0,1
H-LC,77087.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1259104.0      6
           AR         902877.0      5
M-SC       XY24       806028.0      6
H-SC       XR         761610.0      7
X-MC       X40        485608.0      7
           XY24       406047.0      3
           X40N       335078.0      4
H-SC       OX40N      328697.0      4
M-SC       OX40N      320668.0      5
X-SC       X40N       303628.0      3
X-LC       X40        297501.0      5
X-SC       XY24       294641.0      3
H-SC       SR         279324.0      2
X-LC       X40N       217847.0      2
H-MC       AR         210403.0      2
           XY24       192103.0      1
X-SC       X40        184319.0      2
X-LC       XY24       184270.0      2
X-MC       XY25       183242.0      2
M-SC       XR         174505.0      2
L-SC       XR         164674.0      2
M-SC       AR         124652.0      2
M-LC       XR         104678.0      1
X-LC       XY25        90636.0      2
L-SC       OX40N       78090.0      1
H-SC       MH          77564.0      1
H-LC       AR          77087.0      1
L-MC       XR          54005.0      1
M-MC       XY25        50074.0      1
L-LC       XY25        29173.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
